<a href="https://colab.research.google.com/github/alyh/alyh.github.io/blob/master/Validere_Stock_trading_holding_frequency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Validere Data Scientist Interview

### Aly Hassan, Feb 2022


---



## Problem Statement

> **4. Is there an ideal frequency to trade (or hold) stocks as a mom-and-pop investor?**
>
> It is fairly common for people to do some trading with their money, either out of interest or to try to make a profit, whether trading stocks, indices, commodities, crypto assets, etc. Most proprietary traders have some sort of strategy they follow and many even build simple models to guide their trades. The activity levels of at-home traders also varies from person to person. Some are buying and selling every day (or every hour), while others will buy and hold assets for months or years. But which strategy is generally better? Pick any asset class (or even particular asset if you prefer) and, using any relevant data, make a recommendation to an at-home trader as to what frequency of trading will give them most reliable returns for that asset.

The problem, as a I interpret it, can be summarized as follows:

*For an average investor, which of the following maximizes returns: trading often or buying and holding?*


## Approach

First, let's begin by acknowledging that besting the stock market to make the larget return possible is a multi-trillion dollar industry. There are as many rules and philosphies as there are investors. To focus this work, I won't consider stock predictions or stock-picking strategies for now, just general timelines.

Determining how long to hold a stock (days, weeks, months, or years etc.) is a challenge to answer directly because investors generally don't purchase a stock knowing how long they will hold it for. Instead, they purchase a stock thinking it will increase in value *sufficiently* in the next time period, at which point they can sell it and move on to the next stock.

There are exceptions of course; buying around earnings, rumors of acquisitions, and/or product launches that may have a more defined timeline.

To simplify the problem, I will approach the problem by considering the two competing scenarios:

1.   An investor who decides to buy a stock/index once, and will hold it indefinitely.
2.   An investor who decides to buy a stock when it "dips" and sell it when they realize sufficient gain.

The assumption is that the smaller the antipated dip and expected gain, the more often they will trade. For example, if they buy a stock when it dips 5% from its high and plan to sell when their investment is up just 5%, they will be trading a lot more than someone who is waiting for a 10% dip and will only sell when their investment is up 40%.








## Scope & Assumptions

For this analysis, I will focus on the following data:

*   Buying/selling of stocks in the S&P 500 (U.S. Equity Market)
*   Data from Jan 1, 2001 to Dec 31, 2021 (21 years)
*   Data in increments of 1 day


### Assumptions to narrow the scope

1.   Investing based on the "buy low, sell high" swing trading strategy. There are countless stragies, including momentum trading and news trading. Which strategy is best is an entirely different topic. I will focus on "swing trading" in the range of days/weeks vs months vs years by analyzing data on a 1 day period level. This will be compared to not trading at all during the 21 year period.
2.   The 21 year period selected is a representative period of U.S. equity market. Nobody knows for sure if the next 21 years will be anything like the past, but we make the assumption here to guide future decisions. This period starts in the dot-com bubble, experiences the 2007/2008 financial crisis, and finally the historic bull run leading up to COVID-19.
3.   Trading costs are minimal. If you're day trading and have a fee per transaction, this assumption may not hold. Let's assume the investor is using a commission-free broker or are not making enough trades for the fees to be significant.




## Setup & Data

Install & import packages

In [1]:
!pip install pandas
!pip install numpy
!pip install yfinance
!pip install datetime
!pip install pycaret

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

# Approximate list of s&p 500 stocks from public Kaggle dataset: https://www.kaggle.com/camnugent/sandp500
# companies have come and gone, so this list is not 100% accurate for today's market.
#
# I also include 'SPY' which tracks the entire S&P 500
#
sp_500_stocks = ['SPY',
        'MMM','ABT','ABBV','ACN','ATVI','AYI','ADBE','AMD','AAP','AES','AET',
		'AMG','AFL','A','APD','AKAM','ALK','ALB','ARE','ALXN','ALGN','ALLE',
		'AGN','ADS','LNT','ALL','GOOGL','GOOG','MO','AMZN','AEE','AAL','AEP',
		'AXP','AIG','AMT','AWK','AMP','ABC','AME','AMGN','APH','APC','ADI','ANDV',
		'ANSS','ANTM','AON','AOS','APA','AIV','AAPL','AMAT','APTV','ADM','ARNC',
		'AJG','AIZ','T','ADSK','ADP','AZO','AVB','AVY','BHGE','BLL','BAC','BK',
		'BAX','BBT','BDX','BRK.B','BBY','BIIB','BLK','HRB','BA','BWA','BXP','BSX',
		'BHF','BMY','AVGO','BF.B','CHRW','CA','COG','CDNS','CPB','COF','CAH','CBOE',
		'KMX','CCL','CAT','CBG','CBS','CELG','CNC','CNP','CTL','CERN','CF','SCHW',
		'CHTR','CHK','CVX','CMG','CB','CHD','CI','XEC','CINF','CTAS','CSCO','C','CFG',
		'CTXS','CLX','CME','CMS','KO','CTSH','CL','CMCSA','CMA','CAG','CXO','COP',
		'ED','STZ','COO','GLW','COST','COTY','CCI','CSRA','CSX','CMI','CVS','DHI',
		'DHR','DRI','DVA','DE','DAL','XRAY','DVN','DLR','DFS','DISCA','DISCK','DISH',
		'DG','DLTR','D','DOV','DWDP','DPS','DTE','DRE','DUK','DXC','ETFC','EMN','ETN',
		'EBAY','ECL','EIX','EW','EA','EMR','ETR','EVHC','EOG','EQT','EFX','EQIX','EQR',
		'ESS','EL','ES','RE','EXC','EXPE','EXPD','ESRX','EXR','XOM','FFIV','FB','FAST',
		'FRT','FDX','FIS','FITB','FE','FISV','FLIR','FLS','FLR','FMC','FL','F','FTV',
		'FBHS','BEN','FCX','GPS','GRMN','IT','GD','GE','GGP','GIS','GM','GPC','GILD',
		'GPN','GS','GT','GWW','HAL','HBI','HOG','HRS','HIG','HAS','HCA','HCP','HP','HSIC',
		'HSY','HES','HPE','HLT','HOLX','HD','HON','HRL','HST','HPQ','HUM','HBAN','HII',
		'IDXX','INFO','ITW','ILMN','IR','INTC','ICE','IBM','INCY','IP','IPG','IFF','INTU',
		'ISRG','IVZ','IQV','IRM','JEC','JBHT','SJM','JNJ','JCI','JPM','JNPR','KSU','K','KEY',
		'KMB','KIM','KMI','KLAC','KSS','KHC','KR','LB','LLL','LH','LRCX','LEG','LEN','LUK',
		'LLY','LNC','LKQ','LMT','L','LOW','LYB','MTB','MAC','M','MRO','MPC','MAR','MMC','MLM',
		'MAS','MA','MAT','MKC','MCD','MCK','MDT','MRK','MET','MTD','MGM','KORS','MCHP','MU',
		'MSFT','MAA','MHK','TAP','MDLZ','MON','MNST','MCO','MS','MOS','MSI','MYL','NDAQ',
		'NOV','NAVI','NTAP','NFLX','NWL','NFX','NEM','NWSA','NWS','NEE','NLSN','NKE','NI',
		'NBL','JWN','NSC','NTRS','NOC','NCLH','NRG','NUE','NVDA','ORLY','OXY','OMC','OKE',
		'ORCL','PCAR','PKG','PH','PDCO','PAYX','PYPL','PNR','PBCT','PEP','PKI','PRGO','PFE',
		'PCG','PM','PSX','PNW','PXD','PNC','RL','PPG','PPL','PX','PCLN','PFG','PG','PGR',
		'PLD','PRU','PEG','PSA','PHM','PVH','QRVO','PWR','QCOM','DGX','RRC','RJF','RTN','O',
		'RHT','REG','REGN','RF','RSG','RMD','RHI','ROK','COL','ROP','ROST','RCL','CRM','SBAC',
		'SCG','SLB','SNI','STX','SEE','SRE','SHW','SIG','SPG','SWKS','SLG','SNA','SO','LUV',
		'SPGI','SWK','SBUX','STT','SRCL','SYK','STI','SYMC','SYF','SNPS','SYY','TROW','TPR',
		'TGT','TEL','FTI','TXN','TXT','TMO','TIF','TWX','TJX','TMK','TSS','TSLA', 'TSCO','TDG','TRV',
		'TRIP','FOXA','FOX','TSN','UDR','ULTA','USB','UAA','UA','UNP','UAL','UNH','UPS','URI',
		'UTX','UHS','UNM','VFC','VLO','VAR','VTR','VRSN','VRSK','VZ','VRTX','VIAB','V','VNO',
		'VMC','WMT','WBA','DIS','WM','WAT','WEC','WFC','HCN','WDC','WU','WRK','WY','WHR','WMB',
		'WLTW','WYN','WYNN','XEL','XRX','XLNX','XL','XYL','YUM','ZBH','ZION','ZTS']

Use `yfinance` to download prices and volumes for all stocks in our time period. `yfinance` is a tool to use Yahoo's public API and is not affiliated with Yahoo. Data for educational/personal purposes only.

In [3]:
# Download all stocks and store as a pandas dataframe
df_download = yf.download(
    sp_500_stocks,
    start = "2001-01-01",
    end = "2021-12-31",
    interval = "1d",
    progress = True
)

# Save as a parquet for faster loading for future runs
df_download.to_parquet('sp_stocks.parquet')

[*********************100%***********************]  507 of 507 completed

40 Failed downloads:
- DWDP: No data found, symbol may be delisted
- TSS: No data found, symbol may be delisted
- MYL: No data found, symbol may be delisted
- NBL: No data found, symbol may be delisted
- CTL: No data found, symbol may be delisted
- KSU: No data found, symbol may be delisted
- HRS: No data found, symbol may be delisted
- XEC: No data found, symbol may be delisted
- CXO: No data found, symbol may be delisted
- CBG: No data found for this date range, symbol may be delisted
- WYN: No data found for this date range, symbol may be delisted
- RHT: No data found, symbol may be delisted
- UTX: No data found, symbol may be delisted
- ETFC: No data found, symbol may be delisted
- GGP: No data found for this date range, symbol may be delisted
- CBS: No data found, symbol may be delisted
- LB: No data found, symbol may be delisted
- STI: No data found, symbol may be delisted
- HCN: No data found for this date

Some tickers were not found or didn't exist for entire 21 year period. Not a problem, we aren't comparing them against each other.

## Data Processing

Data is fairly clean and usable, but will need to be transformed to simplify our work. Instead of having a column of each ticker, let's transform so that the ticker becomes a column and we can filter on specific tickers. Also, data includes High, Low, Close and Adj Close. Let's only consider adjusted close for the day which is adjusted based on dividends, stocks splits, and other factors.

In [4]:
# data before processing
df_stocks = pd.read_parquet('sp_stocks.parquet')
df_stocks

Adj Close                         ...     Volume                      
                     A        AAL         AAP  ...        ZBH       ZION        ZTS
Date                                           ...                                 
2001-01-02   31.295736        NaN         NaN  ...        NaN   428000.0        NaN
2001-01-03   34.525265        NaN         NaN  ...        NaN   494100.0        NaN
2001-01-04   35.832455        NaN         NaN  ...        NaN   747400.0        NaN
2001-01-05   33.871658        NaN         NaN  ...        NaN  1019700.0        NaN
2001-01-08   32.756710        NaN         NaN  ...        NaN   433200.0        NaN
...                ...        ...         ...  ...        ...        ...        ...
2021-12-23  157.592438  18.260000  232.130005  ...  1396800.0   572000.0  1172400.0
2021-12-27  158.531204  18.170000  236.500000  ...  1185500.0   495500.0   807000.0
2021-12-28  158.970612  18.540001  238.130005  ...  1462700.0   497200.0  1004400.0
2021-12-29  160.438675  18.049999  241.029999  ...  2435700.0   691600.0   939900.0
2021-12-30  160.668396  18.070000  237.520004  ...  1266700.0   413100.0   859100.0

[5291 rows x 3042 columns]

In [5]:
# focus on adjusted close value for each day
df_stocks = df_stocks['Adj Close']

# melt dataset to transform into single ticker column
# reset index so that we get the date as a column instead of the index
df_stocks = df_stocks.melt(var_name='ticker', value_name='close_price', ignore_index=False)
df_stocks = df_stocks.reset_index(drop=False)

# n/a here means the ticker did not exist at a specific date, so there is not price
# let's drop these rows since we won't need them, we are only comparing trading the ticker often
# vs buying once and holding till end of period
df_stocks = df_stocks.dropna(subset=['close_price'])

# print updated dataset
df_stocks

,Date,ticker,close_price
0,2001-01-02,A,31.295736
1,2001-01-03,A,34.525265
2,2001-01-04,A,35.832455
3,2001-01-05,A,33.871658
4,2001-01-08,A,32.756710
...,...,...,...
2682532,2021-12-23,ZTS,242.121170
2682533,2021-12-27,ZTS,246.114746
2682534,2021-12-28,ZTS,243.858383
2682535,2021-12-29,ZTS,246.633926


## Basic Data Exploration

Data is simple, but let's plot it to make sure it makes sense to us

In [7]:
import plotly.express as px

df_stocks_plot = df_stocks[df_stocks['ticker'].isin(['SPY', 'AAPL', 'XOM', 'KMI', 'PXD'])]

fig = px.line(df_stocks_plot, x="Date", y="close_price", color='ticker')
fig.show()

Data makes sense, and we see that some tickers (e.g. KMI in the plot above) start later on, after they had their initial public offering (e.g. KMI was first listed in Feb 2011). The plot is interactive so we can also look at individual stocks and zoom in.

## Scenario Analysis

Now let's begin the analysis.

First, I create a function that can determine when an investor would've bought and sold a stock, given a "buy trigger" and "sell trigger". I define these as follows:
- **Buy Trigger**: Percent drop of a stock from its recent high for it to be purchased, inclusive. Example: 0.05 - Buy stock when it drops >=5% from its recent high.
- **Sell Trigger**: Minimum percent gain for the stock to be sold, inclusive. Note: it might never occur.

So for a given stock, we can determine when the investor would've bought and sold that stock over our 21 year period, and calculate the total gain as well as how long they held the stock on average. 

In [8]:
def stock_scenario_test(df:pd.DataFrame, buy_trigger_perc:float, 
                        sell_trigger_perc:float, date_col:str='Date',
                        price_col:str='close_price'):
    """
        Scenario test that determines buy & sell events for a stock over the given time
        period. Stock is bought when "buy trigger" criteria is met, and sold when "sell trigger"
        criteria is met, if it occurs.

        Parameters:
            df (pd.Dataframe): Dataframe containing the date and price of the stock. Should only
                contain information for one stock.
            buy_trigger_perc (float): Percent drop from previous high that would lead to stock
                being purchased, inclusive. E.g. 0.05 is a 5% drop from high
            sell_trigger_perc (float): Minimum percent gain for the stock to be sold, inclusive.
                May not happen in the given time period, in which case sell price/date will be 
                missing. E.g. 0.25 is 25% gain
            date_col (str):  Name of date column in dataframe, df
            price_col (str): Name of price column in dataframe, df

        Returns:
            df_trades (pd.Dataframe):
                Returns dataframe where every row is a stock purchase, including the buying price.
                For cases when the stock is also sold, it includes the sell price and the sell
                date.
        
    """
    # Make sure data is sorted chronologically
    df = df.sort_values(by=date_col, axis=0, ascending=True)

    # Calculate recent maximum to get percent drop from high
    df['Recent Max'] = df.loc[:, price_col].cummax()
    df['Drop From Max Perc'] = 1 - df[price_col]/df['Recent Max']

    # If drop is greater or equal to our trigger, set a flag to say that this is a potential buy date
    df['Buy Trigger'] = df['Drop From Max Perc'] >= buy_trigger_perc

    # Filter on potential buy dates
    df.loc[df['Buy Trigger'], 'Buy Price'] = df[price_col]
    df_buy_cases = df[df['Buy Trigger']].copy()

    # If we never bought, just return empty dataframe:
    if df_buy_cases.empty:
        return df_buy_cases

    # Our sell target is our buy price * sell trigger (e.g. $30 * 1.1 =  $33 sell price target)
    df_buy_cases['Sell Price Target'] = df_buy_cases['Buy Price'] * (sell_trigger_perc + 1)

    # Helper function that searches the initial dataset for cases when the price of the stock is
    # at or above our target and the date is past our purcahse date
    def sell_condition(x, df, col):
        df_subset = df[(df[date_col] > x[date_col]) & (df[price_col] >= x['Sell Price Target'])]
        if not df_subset.empty:
            return df_subset.iloc[0][col] # get next available day/price
        return np.NaN

    # for all potential buys, get the next possible sell date and sell price, if they exist
    df_buy_cases['Sell Date'] = df_buy_cases.apply(lambda x: sell_condition(x, df, date_col), axis=1)
    df_buy_cases['Sell Price'] = df_buy_cases.apply(lambda x: sell_condition(x, df, price_col), axis=1)

    # get all the cases where we bought and sold a stock
    #
    # actual trades can only occur if the buy date is after the previous sell date, so shift and 
    # find all valid trades
    df_bought_and_sold = df_buy_cases[df_buy_cases[date_col] > df_buy_cases['Sell Date'].shift(fill_value=pd.Timestamp(0))].copy()

    # if we bought once and never sold, return this dataset
    if df_bought_and_sold['Sell Date'].isna().all():
        return df_bought_and_sold

    # if we bought at the end and didn't sell, find the last purchase that we didn't sell
    if df_bought_and_sold['Sell Date'].isna().any():
        # if we sold at least once, get the last sell date and see if we bought again after
        last_sell_date = df_bought_and_sold['Sell Date'].max()
        df_bought_not_sold = df_buy_cases[df_buy_cases['Sell Date'].isna() & (df_buy_cases['Date'] > last_sell_date)]
        
        if not df_bought_not_sold.empty:
            # append the last trade where we bought and didn't sell
            df_bought_and_sold = df_bought_and_sold.append(df_bought_not_sold.iloc[0], ignore_index=True)
            df_bought_and_sold = df_bought_and_sold.drop_duplicates(subset=['Date'])

    # finally, calculate the time we held the stock by subtracting sell date from buy date
    df_bought_and_sold['Time Held'] = df_bought_and_sold['Sell Date'] - df_bought_and_sold[date_col]

    return df_bought_and_sold

### Testing

Let's test a few scenarios to make sure the function is working as intended

In [9]:
# buy GOOG on 10% dip, sell on 30% gain
df_trades = stock_scenario_test(df_stocks[df_stocks['ticker'] == 'GOOG'].copy(), 0.1, 0.3, 'Date', 'close_price')
df_trades

,Date,ticker,close_price,Recent Max,Drop From Max Perc,Buy Trigger,Buy Price,Sell Price Target,Sell Date,Sell Price,Time Held
1090914,2004-11-05,GOOG,84.358803,97.648987,0.136102,True,84.358803,109.666444,2005-04-25,111.347641,171 days
1091111,2005-08-18,GOOG,139.472229,156.383835,0.108142,True,139.472229,181.313898,2005-10-31,185.375168,74 days
1091217,2006-01-20,GOOG,198.984161,234.934402,0.153022,True,198.984161,258.679410,2007-06-22,261.509796,518 days
1091672,2007-11-09,GOOG,330.745270,369.509979,0.104908,True,330.745270,429.968851,2013-05-08,435.183838,2007 days
1093283,2014-04-04,GOOG,541.652893,607.806763,0.108840,True,541.652893,704.148761,2015-10-26,712.780029,570 days
1093732,2016-01-15,GOOG,694.450012,776.599976,0.105782,True,694.450012,902.785016,2017-04-28,905.960022,469 days
1094252,2018-02-05,GOOG,1055.800049,1175.839966,0.102089,True,1055.800049,1372.540063,2020-01-06,1394.209961,700 days
1094771,2020-02-27,GOOG,1318.089966,1526.689941,0.136635,True,1318.089966,1713.516956,2020-09-02,1728.280029,188 days


In [10]:
# buy GOOG on 20% dip, sell on 50% gain
df_trades = stock_scenario_test(df_stocks[df_stocks['ticker'] == 'GOOG'].copy(), 0.2, 0.5, 'Date', 'close_price')
df_trades

,Date,ticker,close_price,Recent Max,Drop From Max Perc,Buy Trigger,Buy Price,Sell Price Target,Sell Date,Sell Price,Time Held
1091229,2006-02-07,GOOG,183.273041,234.934402,0.219897,True,183.273041,274.909561,2007-07-13,275.049042,521 days
1091720,2008-01-22,GOOG,291.083954,369.509979,0.212243,True,291.083954,436.625931,2013-05-10,438.471497,1935 days
1094474,2018-12-20,GOOG,1009.409973,1268.329956,0.204142,True,1009.409973,1514.114960,2020-02-12,1518.270020,419 days
1094778,2020-03-09,GOOG,1215.560059,1526.689941,0.203794,True,1215.560059,1823.340088,2020-12-02,1827.949951,268 days


In [11]:
# buy GOOG on 5% dip, sell on 10% gain
df_trades = stock_scenario_test(df_stocks[df_stocks['ticker'] == 'GOOG'].copy(), 0.05, 0.1, 'Date', 'close_price')
df_trades

,Date,ticker,close_price,Recent Max,Drop From Max Perc,Buy Trigger,Buy Price,Sell Price Target,Sell Date,Sell Price,Time Held
1090866,2004-08-30,GOOG,50.814533,54.495735,0.067550,True,50.814533,55.895987,2004-09-16,56.772205,17 days
1090902,2004-10-20,GOOG,69.982689,74.301498,0.058125,True,69.982689,76.980958,2004-10-22,85.893051,2 days
1090913,2004-11-04,GOOG,92.005142,97.648987,0.057797,True,92.005142,101.205656,2005-01-18,101.569290,75 days
1090956,2005-01-06,GOOG,93.922951,100.976517,0.069854,True,93.922951,103.315246,2005-02-03,105.036301,28 days
1091065,2005-06-14,GOOG,138.655289,146.012711,0.050389,True,138.655289,152.520818,2005-07-19,154.371384,35 days
1091093,2005-07-25,GOOG,147.372604,156.383835,0.057623,True,147.372604,162.109865,2005-10-21,169.315369,88 days
1091182,2005-11-29,GOOG,201.016541,213.509720,0.058513,True,201.016541,221.118195,2006-01-04,221.788681,36 days
1091215,2006-01-18,GOOG,221.624298,234.934402,0.056655,True,221.624298,243.786728,2006-11-15,245.046509,301 days
1091436,2006-12-01,GOOG,239.502289,253.873413,0.056607,True,239.502289,263.452518,2007-06-26,264.139954,207 days
1091593,2007-07-20,GOOG,259.088867,276.463745,0.062847,True,259.088867,284.997754,2007-10-01,290.187317,73 days


In [12]:
# buy KMI on 20% dip, sell on 20% gain
df_trades = stock_scenario_test(df_stocks[df_stocks['ticker'] == 'KMI'].copy(), 0.2, 0.2, 'Date', 'close_price')
df_trades

,Date,ticker,close_price,Recent Max,Drop From Max Perc,Buy Trigger,Buy Price,Sell Price Target,Sell Date,Sell Price,Time Held
0,2011-08-08,KMI,14.423352,18.828070,0.233944,True,14.423352,17.308023,2011-10-21,17.812773,74 days
1,2012-06-07,KMI,19.753521,24.775017,0.202684,True,19.753521,23.704225,2013-01-03,23.920677,210 days
2,2014-03-03,KMI,21.575212,27.101492,0.203911,True,21.575212,25.890255,2014-07-18,25.935303,137 days
3,2015-07-23,KMI,25.241062,31.838398,0.207213,True,25.241062,30.289275,NaT,NaN,NaT


In [13]:
# buy KMI on 20% dip, sell on 200% gain
df_trades = stock_scenario_test(df_stocks[df_stocks['ticker'] == 'KMI'].copy(), 0.2, 2, 'Date', 'close_price')
df_trades

,Date,ticker,close_price,Recent Max,Drop From Max Perc,Buy Trigger,Buy Price,Sell Price Target,Sell Date,Sell Price
1436529,2011-08-08,KMI,14.423352,18.82807,0.233944,True,14.423352,43.270057,NaN,NaN


In [14]:
# buy KMI on 90% dip, sell on 10% gain
df_trades = stock_scenario_test(df_stocks[df_stocks['ticker'] == 'KMI'].copy(), 0.9, 0.1, 'Date', 'close_price')
df_trades

,Date,ticker,close_price,Recent Max,Drop From Max Perc,Buy Trigger,Buy Price


The function is behaving correctly for a variety of edge cases, as tested above:


*   Less trades with wide difference between buy/sell trigger
*   More trades with narrow difference between buy/sell trigger
*   Bought and never sold becaus sell trigger is too high
*   Never bought because buy trigger is too low



---
## Buy & Sell vs Holding

Now we can run our tests and get a recommendation for our average investor

In [15]:
from random import sample

# get 15-20 random stocks, including some hand selected popular stocks (tech, health, banks, energy)
tickers = sample(sp_500_stocks,15)
tickers += ['SPY', 'GOOG', 'AAPL', 'KMI', 'V', 'AMZN', 'XOM', 'PFE', 'JNJ', 'BAC', 'JPM']

# get rid of duplicate tickers
tickers = list(set(tickers))

# buy sell triggers are roughly based on correction (-10%) and major correction (-20%) rules
# these will obviously vary by individual investors
buy_sell_triggers = [(0.1, 0.25), (0.05, 0.2), (0.1, 0.5), (0.2, 0.4), (0.05, 0.1), (0.03, 0.05)]

df_scenarios = pd.DataFrame(columns=['ticker', 'buy_trigger', 'sell_trigger', 'gain',
                                     'time_held_mean', 'time_held_std_dev', 'time_held_median'])

# loop through all tickers and scenarios
for t in tickers:
    for buy, sell in buy_sell_triggers:

        # get trades
        ticker_data = df_stocks[df_stocks['ticker'] == t].copy()
        if ticker_data.empty:
            continue

        df_trades = stock_scenario_test(ticker_data, buy, sell, 'Date', 'close_price')

        # ignore trades where we didn't sell (could be still holding..)
        df_trades = df_trades.dropna()

        if not df_trades.empty:

            # calculate gain over all trades and time held stats
            gain = (df_trades['Sell Price']/df_trades['Buy Price']).prod()

            # calculate time held stats
            time_held_mean = df_trades['Time Held'].dt.days.mean()
            time_held_std = df_trades['Time Held'].dt.days.std()
            time_held_min = df_trades['Time Held'].dt.days.min()
            time_held_max = df_trades['Time Held'].dt.days.max()
            time_held_median = df_trades['Time Held'].dt.days.median()

            df_scenarios = df_scenarios.append({'ticker': t,
                            'buy_trigger': buy,
                            'sell_trigger':  sell,
                            'gain': gain,
                            'time_held_mean': time_held_mean,
                            'time_held_std_dev': time_held_std,
                            'time_held_min': time_held_min,
                            'time_held_max': time_held_max,
                            'time_held_median': time_held_median,
                            }, ignore_index=True)

In [16]:
df_scenarios 

,ticker,buy_trigger,sell_trigger,gain,time_held_mean,time_held_std_dev,time_held_median,time_held_max,time_held_min
0,PFE,0.10,0.25,3.179080,1381.600000,1947.524788,844.0,4800.0,92.0
1,PFE,0.05,0.20,1.827529,2085.333333,2665.829952,1026.0,5118.0,112.0
2,PFE,0.10,0.50,1.500949,5657.000000,NaN,5657.0,5657.0,5657.0
3,PFE,0.20,0.40,1.995721,2723.500000,2831.962659,2723.5,4726.0,721.0
4,PFE,0.05,0.10,3.140558,725.200000,1410.714145,295.5,4682.0,9.0
...,...,...,...,...,...,...,...,...,...
145,NKE,0.05,0.20,17.558323,277.800000,238.858057,192.0,935.0,55.0
146,NKE,0.10,0.50,11.944189,778.666667,233.883447,806.5,1026.0,462.0
147,NKE,0.20,0.40,10.814687,395.142857,213.605890,344.0,699.0,83.0
148,NKE,0.05,0.10,42.447711,162.257143,178.512043,100.0,804.0,2.0


In [17]:
# get the baseline of the gain, if we just held the stock from day 1 until now

df_hold_scenarios = pd.DataFrame(columns=['ticker', 'hold_gain'])

for t in tickers:

    df_ticker_subset = df_stocks[df_stocks['ticker']==t]
    if df_ticker_subset.empty:
        continue

    starting_price = float(df_ticker_subset.iloc[0]['close_price'])
    ending_price = float(df_ticker_subset.iloc[-1:]['close_price'])
    df_hold_scenarios = df_hold_scenarios.append({'ticker': t,
                                                  'hold_gain': ending_price/starting_price,}, ignore_index=True)

df_hold_scenarios

,ticker,hold_gain
0,PFE,2.795862
1,NWL,1.727480
2,JNJ,5.836205
3,AAPL,780.170063
4,XOM,2.628962
5,GM,2.222475
6,CPB,2.274353
7,DVA,21.157353
8,RSG,19.041456
9,WU,1.340739


In [18]:
# merge baseline gain (called hold_gain) and calculate our scenario's relative performance
df_scenarios = df_scenarios.merge(right=df_hold_scenarios, on='ticker', how='left')
df_scenarios['relative_gain'] = df_scenarios['gain']/df_scenarios['hold_gain']
df_scenarios

,ticker,buy_trigger,sell_trigger,gain,time_held_mean,time_held_std_dev,time_held_median,time_held_max,time_held_min,hold_gain,relative_gain
0,PFE,0.10,0.25,3.179080,1381.600000,1947.524788,844.0,4800.0,92.0,2.795862,1.137066
1,PFE,0.05,0.20,1.827529,2085.333333,2665.829952,1026.0,5118.0,112.0,2.795862,0.653655
2,PFE,0.10,0.50,1.500949,5657.000000,NaN,5657.0,5657.0,5657.0,2.795862,0.536847
3,PFE,0.20,0.40,1.995721,2723.500000,2831.962659,2723.5,4726.0,721.0,2.795862,0.713812
4,PFE,0.05,0.10,3.140558,725.200000,1410.714145,295.5,4682.0,9.0,2.795862,1.123288
...,...,...,...,...,...,...,...,...,...,...,...
145,NKE,0.05,0.20,17.558323,277.800000,238.858057,192.0,935.0,55.0,74.657103,0.235186
146,NKE,0.10,0.50,11.944189,778.666667,233.883447,806.5,1026.0,462.0,74.657103,0.159987
147,NKE,0.20,0.40,10.814687,395.142857,213.605890,344.0,699.0,83.0,74.657103,0.144858
148,NKE,0.05,0.10,42.447711,162.257143,178.512043,100.0,804.0,2.0,74.657103,0.568569


In [19]:
# drop cases where we had a single trade
df_scenarios = df_scenarios.dropna(subset=['time_held_std_dev'])

# Plot relative gain vs median time held (indication of how often we're trading)
# The size is the absolute baseline gain to show the magnitude of some scenarios over others
# i.e. a very large circle near the bottom of y-axis is a huge missed oppurtunity

fig = px.scatter(df_scenarios, x="time_held_median", y="relative_gain", color='ticker', size='hold_gain',
                 log_x=True, size_max=75, 
                 labels={
                     "time_held_median": "Median Time Held (days)",
                     "relative_gain": "Relative Gain Ratio",
                     "ticker": "Ticker",
                     "hold_gain": "Baseline Gain if Held"
                 },)
fig.show()


In [20]:
# Understand the uncertainty in the time held by plotting the average along with std dev as
# the error. Huge error bars show that time held is uncertain as market is unpredictable.

fig = px.scatter(df_scenarios, y="time_held_mean", x="relative_gain", color='ticker', 
                 error_y="time_held_std_dev",
                 labels={
                     "time_held_mean": "Mean Time Held (days)",
                     "relative_gain": "Relative Gain Ratio",
                     "ticker": "Ticker"
                 },)
fig.show()

In [21]:
# Plot just SPY (S&P 500) performance for our scenarios, which avoids question of stock picking

fig = px.bar(df_scenarios[df_scenarios['ticker']=='SPY'], y="relative_gain", x="time_held_mean",
                 labels={
                     "time_held_mean": "Mean Time Held (days)",
                     "relative_gain": "Relative Gain for S&P 500 index",
                     "ticker": "Ticker"
                 },)
fig.show()

## Conclusions & Recommendation

Before making the recommendations, let's acknowledge the limitations of this analysis:


*   There is not enough data to directly correlate "time held" to relative peformance. There may be no correlation. We would need more time data and many more stocks to draw a final conclusion.
*   We only considered 15-20 stocks and the SPY index for 21 years. By selecting large U.S. companies over a 21-year period, we are trying to find an acccurate represtative sample. The results may or may not apply for future years or other stock/equity/commodity markets.



### Conclusions



1.   In general, it doesn't really matter if you hold a stock for a few days, weeks, or months. Trying to time the market almost always leads to equivalent or worse performance compared to just buying and holding once in that same time period. There are exceptions of course. The challenge is knowing which stock to buy, when to cut losses, and how far up a stock could rise. In reality, we know none of those.
2.   There is huge uncertainty around how long you will end up holding a stock depending on your buy/sell conditions. Once again, this is due to market uncertainty and not knowing how the stock will perform after you buy it, if you have a minimum expected gain.


### Recommendation

Unless you know something that the rest of the market doesn't, it's better to just buy and hold (10+ years) an index/stock than to actively trade, whether that's day-to-day trading or month-to-month trading. For S&P500 index, holding for the full 21 years in our example performs better than active trading for all scenarios. The same is true for other popular, top-performing stocks like Google, Amazon, Visa and JP Morgan.

For more proof, look no further than the multi-million dollar bet that Warren Buffet made in 2007. A bet that the S&P 500 would outperform the active trading hedge funds for the next 10 years. He won: https://www.cnbc.com/2018/02/16/warren-buffett-won-2-point-2-million-on-a-bet-and-gave-it-to-girls-inc.html




# Extra - Why can't we predict price?

Building a proper stock predicting model is out-of-scope, but I build a very basic one here with the help of pycaret's time series module to illustrate the challenges.

In [22]:
# can't use datetime directly, so create date feaures
df_stocks['month'] = df_stocks['Date'].dt.month
df_stocks['year'] = df_stocks['Date'].dt.year
df_stocks['day_of_week'] = df_stocks['Date'].dt.dayofweek
df_stocks['day_of_year'] = df_stocks['Date'].dt.dayofyear

# focus on SPY index
df_spy = df_stocks[df_stocks['ticker']=='SPY'].copy()

# create day series (1 to n where n is number of days in our 21 year period)
df_spy = df_spy.reset_index(drop=True)
df_spy['day'] = df_spy.index

# split data into train-test set (roughly 85/15 - more weight on train)
train = df_spy[df_spy['year'] < 2018]
test = df_spy[df_spy['year'] >= 2019]

train

,Date,ticker,close_price,month,year,day_of_week,day_of_year,day
0,2001-01-02,SPY,87.268913,1,2001,1,2,0
1,2001-01-03,SPY,91.460861,1,2001,2,3,1
2,2001-01-04,SPY,90.476395,1,2001,3,4,2
3,2001-01-05,SPY,87.522957,1,2001,4,5,3
4,2001-01-08,SPY,88.200439,1,2001,0,8,4
...,...,...,...,...,...,...,...,...
4271,2017-12-22,SPY,251.273697,12,2017,4,356,4271
4272,2017-12-26,SPY,250.973114,12,2017,1,360,4272
4273,2017-12-27,SPY,251.095245,12,2017,2,361,4273
4274,2017-12-28,SPY,251.611847,12,2017,3,362,4274


In [23]:
from pycaret.regression import *

# setup basic times series model with pycaret
s = setup(data = train, 
          test_data = test, 
          target = 'close_price', 
          fold_strategy = 'timeseries', 
          ignore_features=['Date', 'ticker'],
          numeric_features = ['year', 'day'],
          categorical_features = ['day_of_week','day_of_year','month'],
          fold = 3, 
          transform_target = True, # transform into normal shape 
          session_id = 123,
          verbose=False)

best = compare_models(sort = 'RMSE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,31.9068,1723.0298,34.5206,-2.4371,0.2609,0.2141,0.2867
omp,Orthogonal Matching Pursuit,32.0719,1729.5350,34.6824,-2.4637,0.2637,0.2160,0.0833
ridge,Ridge Regression,32.3856,1758.6918,34.9839,-2.5157,0.2670,0.2181,0.0833
rf,Random Forest Regressor,35.0744,1788.0808,39.0137,-3.1528,0.3302,0.2555,2.7800
et,Extra Trees Regressor,35.2400,1794.6236,39.1513,-3.1731,0.3316,0.2569,3.3833
dt,Decision Tree Regressor,35.3450,1802.0138,39.2430,-3.2048,0.3336,0.2583,0.1133
ada,AdaBoost Regressor,35.7199,1862.1151,39.6011,-3.2450,0.3316,0.2576,0.4700
lightgbm,Light Gradient Boosting Machine,36.3336,1891.4934,40.0987,-3.3961,0.3423,0.2653,0.2467
en,Elastic Net,36.7630,2243.5912,40.1212,-3.4783,0.3101,0.2426,0.0767
gbr,Gradient Boosting Regressor,37.1152,1965.0170,40.7895,-3.5606,0.3514,0.2712,1.1300


In [24]:
# get metrics on test set
test_set_results = predict_model(best);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Bayesian Ridge,77.0804,7767.024,88.1307,-0.7258,0.2632,0.2117


In [25]:
# plot train and test regions of the data (price vs predictions)

predictions = predict_model(best, data=train)
predictions = predictions.rename({"Label":"close_price_predicted"}, axis=1)
fig = px.line(predictions, x='Date', y=["close_price", "close_price_predicted"])
fig.show()

predictions = predict_model(best, data=test)
predictions = predictions.rename({"Label":"close_price_predicted"}, axis=1)
fig = px.line(predictions, x='Date', y=["close_price", "close_price_predicted"])
fig.show()

## Thoughts & Conclusions on Modelling

1. Model is incredibly overfit. The test RMSE is 88 vs train RMSE of 34. The MAE of the test set is 77 - that's almost 20% of the current SPY price. This is common with stock predicting models, as it's very difficult to learn from past patterns to predict future events. Many models learn the noise rather than the underlying cause. For example, if a new crash was caused by a sudden news report, how could the model know?

2. Model is not very predictive in general and needs many more features. These may include engineered features like moving average, 52-week high, or momentum. Additional features like volume, price-to-earnings ratio, debt, interest rates, and more will also benefit the model. Features relating to outside factors like pandemics, war, company products, and consumer sentiment are much more difficult to quantify and capture.

3. If an average investor is using a model to guide active trading decisions, this illustrates how important it is to make sure that the model is generalizable. And if it's not, buy and hold.
